In [3]:
# основные библиотеки для работы с данными
import pandas as pd
import numpy as np
import re

# инструменты визуализации
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
pd.set_option('display.max_columns', 30)

In [5]:
df_total = pd.read_csv('/Users/alex_eyfler/ds_bootcamp/rent_real_estate_team_pandas/Alexander/data/data.csv')

In [7]:
# взял верхнюю половину, Ламберт, не забудь изменить на нижнюю :)
df = df_total.iloc[0:11684, :]
df.head()

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


In [8]:
# Сразу же дропаем колонку с дублированными индексами
df = df.drop(labels='Unnamed: 0', axis=1)

In [9]:
# удалим полные дубликаты строк по значимым параметрам
# можно предположить, что остальные параметры зависят от выбранных или могут
# быть не заполнены в повторных объявлениях. Если происходит ситуация, что два
# или более объявления на одну квартиру (риэлтор и собственник незвисимо)
# выставили, выставили новое объявление, изменив цену, но забыв удалить старое и тд

## в изначальной половине датасета было 80 полных дубликатов по выбранному
## сабсету, их и удалили

df = df.drop_duplicates(subset=[
    'Количество комнат',
    'Адрес',
    'Площадь, м2',
    'Дом'
])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11604 entries, 0 to 11683
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID  объявления            11604 non-null  int64  
 1   Количество комнат         11193 non-null  object 
 2   Тип                       11604 non-null  object 
 3   Метро                     11077 non-null  object 
 4   Адрес                     11604 non-null  object 
 5   Площадь, м2               11604 non-null  object 
 6   Дом                       11604 non-null  object 
 7   Парковка                  5009 non-null   object 
 8   Цена                      11604 non-null  object 
 9   Телефоны                  11604 non-null  object 
 10  Описание                  11604 non-null  object 
 11  Ремонт                    10223 non-null  object 
 12  Площадь комнат, м2        7275 non-null   object 
 13  Балкон                    7554 non-null   object 
 14  Окна       

In [10]:
# уберем столбцы, которые в явном виде не несут никакого смысла для целей
# исследования

df = df.drop(labels=[
    'Тип',
    'Адрес',
    'Телефоны',
    'Описание',
    'Дополнительно',
    'Название ЖК',
    'Серия дома',
    'Ссылка на объявление'
], axis=1)

In [11]:
# получим количество комнат в виде int числа
# для этого напишем функцию для отделения первого числа из ячейки
# но на самом деле будет float пока что, т.к. NaN не заполняем по условию первого коммита

## РАЗОБРАТЬСЯ С РЕГУЛЯРНЫМИ ВЫРАЖЕНИЯМИ

def first_digit(value):
    """
    Находим количество комнат, отбрасывая текст и почие комментарии, оставляя
    только число

    Args:
        value (str or NaN): значение в конкретной ячейке

    Returns:
        int or NaN: Если был NaN, возращаем его, если нет - число комнат
    """
    if pd.isna(value):
        return np.nan
    match = re.match(r'(\d+)', value)
    if match:
        return int(match.group(1))

# перезаписываем колонку количества комнат
df['Количество комнат'] = df['Количество комнат'].apply(first_digit)
    

In [12]:
# обработаем колонку метро

df['Метро'] = [np.nan if pd.isna(row) else row.split(' (')[0] for row in df['Метро']]
df['Метро'] = [np.nan if pd.isna(row) else row.split('. ')[1] for row in df['Метро']]

In [13]:
# обработаем колонку площадь
# оставим самое большое значение, т.к. логично предположить, что это общая площадь

def split_square(value):
    """
    Функция для обработки столбца с площадью

    Args:
        value (str or NaN): значение в конкретной ячейке

    Returns:
        float or NaN: максимальное значение площади или NaN, если было NaN
    """
    if pd.isna(value):
        return np.nan
    answer = list(map(float, value.split('/')))
    return max(answer)

# создадим новую колонку - общая площадь квартиры
df['Общая площадь, м2'] = df['Площадь, м2'].apply(split_square)

# удалим старую колонку
df.drop(labels=['Площадь, м2', 'Площадь комнат, м2'], axis=1, inplace=True)

In [14]:
# обработаем колонку дом
# возьмем из него: этаж квартиры, общая этажность дома, тип дома

df['Тип дома'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
df['Этажность дома'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['Этаж квартиры'] = [int(row.split('/')[0]) for row in df['Дом']]

# удалим старую колонку дом
df.drop(labels='Дом', axis=1, inplace=True)

In [15]:
# обрабатываем колонку балкон
# вот тут было бы удобнее сразу заменить пропущенные значения нулями
# но раз по ТЗ в первом коммите мы не заменяем NaNы, приходится так

def find_balcony(value):
    """
    Ищет в ячейке балкон и возвращает количество балконов

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - кол-во балконов
    """
    if pd.isna(value):
        return np.nan
    if value.split(' ')[0] == 'Балкон':
        return value.split(' ')[1]
    return '(0)'

def find_loggia(value):
    """
    Ищет в ячейке лоджию и возвращает количество лоджий

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - кол-во лоджий
    """
    if pd.isna(value):
        return np.nan
    if value.split(' ')[-2] == 'Лоджия':
        return value.split(' ')[-1]
    return '(0)'

# создаем новую колонку лоджии и перезаписываем колонку балкона
df['Лоджия'] = df['Балкон'].apply(find_loggia)
df['Балкон'] = df['Балкон'].apply(find_balcony)

In [16]:
# очистим данные по балкону и лоджии, уберя лишние символы и преобразуя в тип int

df['Лоджия'] = [np.nan if pd.isna(row) else row.split('(')[1] for row in df['Лоджия']]
df['Лоджия'] = [np.nan if pd.isna(row) else int(row.split(')')[0]) for row in df['Лоджия']]
df['Балкон'] = [np.nan if pd.isna(row) else row.split('(')[1] for row in df['Балкон']]
df['Балкон'] = [np.nan if pd.isna(row) else int(row.split(')')[0]) for row in df['Балкон']]

In [17]:
# обработаем колонку санузел

def find_combined(value):
    """
    Ищет в ячейке совмещенный санузел и возвращает количество

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - кол-во совмещенных
        санузлов
    """
    if pd.isna(value):
        return np.nan
    if value.split(' ')[0] == 'Совмещенный':
        return value.split(' ')[1]
    return '(0)'

def find_separated(value):
    """
    Ищет в ячейке раздельный санузел и возвращает количество

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - кол-во раздельных
        санузлов
    """
    if pd.isna(value):
        return np.nan
    if value.split(' ')[-2] == 'Раздельный':
        return value.split(' ')[-1]
    return '(0)'

# создаем новые колонки для совмещенных и раздельных санузлов
df['Санузел совмещенный'] = df['Санузел'].apply(find_combined)
df['Санузел раздельный'] = df['Санузел'].apply(find_separated)

In [18]:
# очистим данные по санузлу, уберя лишние символы и преобразуя в инт
# но опять неудобно! преобразовать в int столбец нельзя, т.к. NaN еще не заполнены!!!

df['Санузел совмещенный'] = [np.nan if pd.isna(row) else row.split('(')[1] for row in df['Санузел совмещенный']]
df['Санузел совмещенный'] = [np.nan if pd.isna(row) else int(row.split(')')[0]) for row in df['Санузел совмещенный']]
df['Санузел раздельный'] = [np.nan if pd.isna(row) else row.split('(')[1] for row in df['Санузел раздельный']]
df['Санузел раздельный'] = [np.nan if pd.isna(row) else int(row.split(')')[0]) for row in df['Санузел раздельный']]

# удаляем старую колонку санузел
df.drop(labels='Санузел', axis=1, inplace=True)

In [19]:
# обработаем колонку лифт, не очищая от NaNов

def find_passangers(value):
    """
    Ищет в ячейке пассажирский лифт и возвращает количество

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - кол-во пассажирских
        лифтов
    """
    if pd.isna(value):
        return np.nan
    if value.split(' ')[0] == 'Пасс':
        return value.split(' ')[1]
    return '(0)'

def find_cargo(value):
    """
    Ищет в ячейке грузовой лифт и возвращает количество

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - кол-во грузовых
        лифтов
    """
    if pd.isna(value):
        return np.nan
    if value.split(' ')[-2] == 'Груз':
        return value.split(' ')[-1]
    return '(0)'

# создаем новые колонки для пассажирских и грузовых лифтов
df['Лифт пасс'] = df['Лифт'].apply(find_passangers)
df['Лифт груз'] = df['Лифт'].apply(find_cargo)

In [20]:
# очистим данные по санузлу, уберя лишние символы и преобразуя в инт
# но опять неудобно! преобразовать в int столбец нельзя, т.к. NaN еще не заполнены!!!

df['Лифт пасс'] = [np.nan if pd.isna(row) else row.split('(')[1] for row in df['Лифт пасс']]
df['Лифт пасс'] = [np.nan if pd.isna(row) else int(row.split(')')[0]) for row in df['Лифт пасс']]
df['Лифт груз'] = [np.nan if pd.isna(row) else row.split('(')[1] for row in df['Лифт груз']]
df['Лифт груз'] = [np.nan if pd.isna(row) else int(row.split(')')[0]) for row in df['Лифт груз']]

# удлаяем старую колонку лифт
df.drop(labels='Лифт', axis=1, inplace=True)

In [21]:
# обработаем столбец можно с детьми/животными

def kids(value):
    """
    Ищет в ячейке Можно с детьми и возвращает 1, если да

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - 1 при обнаружении
        искомой фразы
    """
    if pd.isna(value):
        return np.nan
    if 'Можно с детьми' in value:
        return 1

def pets(value):
    """
    Ищет в ячейке Можно с животными и возвращает 1, если да

    Args:
        value (str or NaN): значение в ячейке

    Returns:
        str or NaN: Если был NaN, возращаем его, если нет - 1 при обнаружении
        искомой фразы
    """
    if pd.isna(value):
        return np.nan
    if 'Можно с животными' in value:
        return 1

# создаем новые колонки можно с детьми и можно с животными    
df['Можно с детьми'] = df['Можно с детьми/животными'].apply(kids)
df['Можно с животными'] = df['Можно с детьми/животными'].apply(pets)

# удаляем старую колонку можно с детьми/животными
df.drop(labels='Можно с детьми/животными', axis=1, inplace=True)

In [22]:
# работаем с ценой
# приняли решение просто отрезать цену и взять цену аренды, не учитывая коммуналку
# залог и дополнительные параметры, т.к. данные слишком грязные, структура записей
# в колонце не унифицирована

# создаем новую колонку цена аренды
df['Цена аренды'] = [float(row.split(' ')[0]) for row in df['Цена']]

# удаляем старую колонку цена
df.drop(labels='Цена', axis=1, inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11604 entries, 0 to 11683
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID  объявления       11604 non-null  int64  
 1   Количество комнат    11193 non-null  float64
 2   Метро                11077 non-null  object 
 3   Парковка             5009 non-null   object 
 4   Ремонт               10223 non-null  object 
 5   Балкон               7554 non-null   float64
 6   Окна                 8372 non-null   object 
 7   Высота потолков, м   5998 non-null   float64
 8   Мусоропровод         6614 non-null   object 
 9   Общая площадь, м2    11604 non-null  float64
 10  Тип дома             11604 non-null  object 
 11  Этажность дома       11604 non-null  int64  
 12  Этаж квартиры        11604 non-null  int64  
 13  Лоджия               7554 non-null   float64
 14  Санузел совмещенный  10284 non-null  float64
 15  Санузел раздельный   10284 non-null  floa

In [24]:
df.head()

,ID объявления,Количество комнат,Метро,Парковка,Ремонт,Балкон,Окна,"Высота потолков, м",Мусоропровод,"Общая площадь, м2",Тип дома,Этажность дома,Этаж квартиры,Лоджия,Санузел совмещенный,Санузел раздельный,Лифт пасс,Лифт груз,Можно с детьми,Можно с животными,Цена аренды
0,271271157,4.0,Смоленская,подземная,Дизайнерский,NaN,NaN,3.0,Да,200.0,Монолитный,16,5,NaN,NaN,NaN,4.0,1.0,1.0,1.0,500000.0
1,271634126,4.0,Смоленская,подземная,Дизайнерский,NaN,На улицу и двор,3.5,Нет,198.0,Монолитно-кирпичный,16,5,NaN,2.0,1.0,1.0,1.0,1.0,NaN,500000.0
2,271173086,4.0,Смоленская,подземная,Евроремонт,NaN,На улицу и двор,3.2,NaN,200.0,Неизвестно,16,5,NaN,3.0,0.0,1.0,0.0,1.0,NaN,500000.0
3,272197456,4.0,Смоленская,подземная,Евроремонт,NaN,На улицу и двор,3.2,NaN,170.0,Неизвестно,6,5,NaN,3.0,0.0,1.0,0.0,NaN,1.0,400000.0
4,273614615,2.0,Арбатская,NaN,Евроремонт,NaN,На улицу и двор,3.9,Да,58.0,Панельный,26,12,NaN,2.0,0.0,1.0,1.0,NaN,NaN,225000.0


In [25]:
df.to_csv('/Users/alex_eyfler/ds_bootcamp/rent_real_estate_team_pandas/Alexander/data/cleaned_data.csv', index=False)